# Wellington房产预测模型 - 基于真实数据

<a href="https://colab.research.google.com/github/NZLouislu/nzlouis-property-ai-engine/blob/main/notebooks/Wellington_Property_Prediction_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

这个notebook实现了一个高精度的Wellington房产预测模型，从real_estate表获取真实数据进行训练，能够预测房产是否适合出售。

## 🚀 快速开始
点击上方的 **"Open in Colab"** 按钮，然后选择 **"运行时" → "全部运行"** 即可开始！

## 主要功能
- 从real_estate表获取真实训练数据
- 高级特征工程
- 集成学习模型(随机森林+梯度提升+逻辑回归)
- 生成高置信度Wellington房产预测
- 自动保存预测结果到数据库

## 1. 环境设置和依赖安装

In [ ]:
# 安装必要的包
!pip install pandas numpy scikit-learn joblib matplotlib seaborn supabase python-dotenv

import sys
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
import joblib
from datetime import datetime
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import json
from dotenv import load_dotenv
from supabase import create_client, Client
import uuid

warnings.filterwarnings("ignore")
print("✅ 所有依赖包安装完成")

## 2. 上传必要文件

在Colab中，我们需要上传以下文件：
1. `wellington_real_data_predictor.py` - 包含从real_estate表获取数据的函数
2. `.env` 文件 - 包含Supabase凭据

In [ ]:
# 上传wellington_real_data_predictor.py文件
from google.colab import files

print("请上传wellington_real_data_predictor.py文件...")
uploaded = files.upload()

if 'wellington_real_data_predictor.py' in uploaded:
    print("✅ wellington_real_data_predictor.py上传成功")
else:
    print("❌ 未找到wellington_real_data_predictor.py文件，请重新上传")

In [ ]:
# 上传.env文件
print("请上传.env文件（包含SUPABASE_URL和SUPABASE_KEY）...")
uploaded = files.upload()

if '.env' in uploaded:
    print("✅ .env文件上传成功")
    # 加载环境变量
    load_dotenv()
else:
    print("⚠️ 未找到.env文件，请手动设置环境变量")
    # 手动设置环境变量
    # os.environ['SUPABASE_URL'] = 'your_supabase_url_here'
    # os.environ['SUPABASE_KEY'] = 'your_supabase_key_here'

## 3. 导入wellington_real_data_predictor.py中的函数

In [ ]:
# 导入wellington_real_data_predictor.py中的所有函数
from wellington_real_data_predictor import *

# 创建Supabase客户端连接
supabase_client = create_supabase_client()
if supabase_client:
    print("✅ Supabase连接成功！")
else:
    print("❌ Supabase连接失败，请检查.env文件中的凭据")

## 4. 从real_estate表获取数据

In [ ]:
print("🔄 从real_estate表获取训练数据...")
data = get_training_data_from_real_estate(supabase_client)
if data is not None:
    print(f"✅ 成功加载 {len(data)} 条记录")
    print("\n📊 数据形状:", data.shape)
    print("\n📋 列名:")
    print(data.columns.tolist())
    
    # 显示数据样本
    print("\n📊 数据样本:")
    display(data.head())
else:
    print("❌ 无法获取数据，请检查数据库连接和表结构")

## 5. 数据处理和特征工程

In [ ]:
print("🔄 处理数据...")
# 处理real_estate表的数据
training_data, available_columns = process_real_estate_data(data)
if training_data is not None:
    print(f"✅ 数据处理完成，共 {len(training_data)} 条记录")
    print(f"📋 可用列: {available_columns}")
    
    # 创建特征
    X, feature_names = create_features_from_real_data(training_data)
    y = training_data['target'].values
    
    print(f"\n📊 训练数据统计:")
    print(f"  样本数量: {len(X)}")
    print(f"  特征数量: {len(feature_names)}")
    print(f"  标签分布: {pd.Series(y).value_counts().to_dict()}")
    
    processed_data = X.copy()
    processed_data['target'] = y
    
    # 显示处理后的数据
    print("\n📊 处理后的数据样本:")
    display(processed_data.head())
else:
    print("❌ 数据处理失败")